### Este código procesa los resultados de las cuatro iteraciones de BP y sus correspondientes valores de BBP. Para cada conjunto, calcula el promedio y el error estándar, que son necesarios para determinar los intervalos de confianza con un nivel de confianza predefinido.

#### Funciones

In [24]:
# Read files
import os
import sys

# This function returns each line of a file as a list
def read_file(file_name):
    with open(file_name, 'r') as file:
        return file.readlines()

# Extract the value after "//" in a line
def extract_value(line):
    stripped = line.split('\\\\')  # Corrected to split by '//'
    if len(stripped) > 1:
        return stripped[1].strip()
    else:
        print(line)
        return None

# Save values in list of lists
def get_values(file_name):
    values = []
    # Map of values
    map_values = {}
    counter = 0
    for line in read_file(file_name):
        if "BBP RESULTS" in line:  # Detener el procesamiento al encontrar "BBP RESULTS"
            break
        if line.strip():  # Saltar líneas vacías
            if line[:4] not in map_values:
                map_values[line[:4]] = counter
                values.append([])
                counter += 1
            add_value = extract_value(line)
            if add_value is not None:
                try:
                    values[map_values[line[:4]]].append(float(add_value))
                except:
                    print("Error in line: ", line)
    return values, list(map_values.keys())


In [25]:
import numpy as np
from scipy.stats import t

def calculate_statistics(data, confidence_level=0.95):
    """
    Calcula la desviación estándar, la media, el error estándar y los intervalos de confianza
    de una lista de datos.
    
    Parámetros:
    - data (list): Una lista de números.
    - confidence_level (float): El nivel de confianza deseado (por defecto, 0.95).
    
    Retorna:
    - dict: Un diccionario con los siguientes valores:
        - 'mean': La media de los datos.
        - 'std_dev': La desviación estándar de los datos.
        - 'std_error': El error estándar de la media.
        - 'confidence_interval': Una tupla con los límites inferior y superior del intervalo de confianza.
    """
    n = len(data)
    
    # Verificar si todos los valores son cero
    if all(x == 0 for x in data):
        return {
            "mean": 0,
            "std_dev": 0,
            "std_error": 0,
            "margin_of_error": 0
        }
    
    # Cálculos estadísticos
    mean = np.mean(data)
    std_dev = np.std(data, ddof=1) if n > 1 else 0  # Desviación estándar muestral
    std_error = std_dev / np.sqrt(n) if n > 1 else 0
    
    # Calcular el intervalo de confianza
    if n > 1:
        t_critical = t.ppf((1 + confidence_level) / 2, df=n - 1)
        margin_of_error = t_critical * std_error
    else:
        # Si solo hay un valor, el intervalo de confianza no tiene sentido
        margin_of_error = 0
    
    return {
        "mean": mean,
        "std_dev": std_dev,
        "std_error": std_error,
        "margin_of_error": margin_of_error
    }

#### Ejecución

In [26]:
# Lista de los archivos en el directorio actual
files = os.listdir()
files.sort()

In [27]:
# Iterar sobre los archivos y calcular las estadísticas (excepto en el archivo actual y en los resultados de incremental)
for i,file in enumerate(files):
    print(file)
    if file == 'compute_BBP.ipynb' or file == 'results_INCREMENTAL.csv':
        continue
    values, map_values = get_values(file)
    bands = file.split('_')
    with open(file, 'a') as filee:
        filee.write("BBP RESULTS\n")
        for j, value in enumerate(values):
            statistics = calculate_statistics(value)
            filee.write(f"{map_values[j]}\t{statistics['mean']}\t{float(statistics['margin_of_error'])}\t{float(statistics['margin_of_error'])}\\\\\n")

EuroCore_CLE_BestFit.csv
EuroCore_CLE_FirstFit.csv
EuroCore_CLS_BestFit.csv
EuroCore_CLS_FirstFit.csv
EuroCore_CL_125_BestFit.csv
EuroCore_CL_125_FirstFit.csv
EuroCore_CL_50_BestFit.csv
EuroCore_CL_50_FirstFit.csv
EuroCore_CL_625_BestFit.csv
EuroCore_CL_625_FirstFit.csv
NSFNet_CLE_BestFit.csv
NSFNet_CLE_FirstFit.csv
NSFNet_CLS_BestFit.csv
NSFNet_CLS_FirstFit.csv
NSFNet_CL_125_BestFit.csv
NSFNet_CL_125_FirstFit.csv
NSFNet_CL_50_BestFit.csv
NSFNet_CL_50_FirstFit.csv
NSFNet_CL_625_BestFit.csv
NSFNet_CL_625_FirstFit.csv
UKNet_CLE_BestFit.csv
UKNet_CLE_FirstFit.csv
UKNet_CLS_BestFit.csv
UKNet_CLS_FirstFit.csv
UKNet_CL_125_BestFit.csv
UKNet_CL_125_FirstFit.csv
UKNet_CL_50_BestFit.csv
UKNet_CL_50_FirstFit.csv
UKNet_CL_625_BestFit.csv
UKNet_CL_625_FirstFit.csv
compute_BBP.ipynb
results_INCREMENTAL.csv
